<u>Observations</u>: 

1) closest distance to line
2) heading angle relative to line
3) road curvature at closest point
4) vx sensor
5) heading angular rate gyro
6) look ahead road curvatures

<u>Reward Components</u>:

1) road progress reward
2) bounded piecewise quadratic road proximity reward (positive reward conditional on road progress)
3) bounded piecewise quadratic-linear speed reward with adaptive speed limit based on road curvature multiplier (positive reward conditional on road progress)
4) bounded piecewise heading angle reward
5) angular velocity smoothness penalty

<u>Model</u>:

SAC + Modified Neural Net:

policy_kwargs = dict(net_arch=[256, 256, 256])

`road_randomization_params = {
    'num_elements_range': (3, 10),             # Highly variable number of road elements
    'straight_length_range': (50.0, 300.0),    # Straight segments can be very short or long
    'curvature_range': (-1/50.0, 1/50.0),      # Very tight curves possible
    'angle_range': (10.0, 150.0)                 # Wide range of angles for curves   
}`


* High-speed road with speed limit 100 km/h


## Library Imports:

In [ ]:
import ai4rgym
from ai4rgym.envs.road import Road
import gymnasium as gym
import numpy as np
import random
from stable_baselines3 import PPO, SAC, DDPG
from utils import ensure_dir, ensure_dirs, eval_model
import matplotlib.pyplot as plt

## Environment Settings:

In [ ]:
# SPECIFY THE VEHCILE PARAMETERS
bicycle_model_parameters = {
    "Lf" : 0.55*2.875,
    "Lr" : 0.45*2.875,
    "m"  : 2000.0,
    "Iz" : (1.0/12.0) * 2000.0 * (4.692**2+1.850**2),
    "Cm" : (1.0/100.0) * (1.0 * 400.0 * 9.0) / 0.2286,
    "Cd" : 0.5 * 0.24 * 2.2204 * 1.202,
    "delta_offset" : 0 * np.pi/180,
    "delta_request_max" : 45 * np.pi/180,
    "Ddelta_lower_limit" : -45 * np.pi/180,
    "Ddelta_upper_limit" :  45 * np.pi/180,
    "v_transition_min": 3.0,    # v_transition_min = 3.0 m/s
    "v_transition_max": 5.0,    # v_transition_max = 5.0 m/s
    "body_len_f" : (0.55*2.875) * 1.5,
    "body_len_r" : (0.45*2.875) * 1.5,
    "body_width" : 2.50,
}

# SPECIFY THE ROAD

# This road will not be used for training / evaluation
# this is just kept in place to initialize the environment
road_elements_list = [
    {"type":"straight", "length":100.0},
    {"type":"curved", "curvature":1/800.0, "angle_in_degrees":15.0},
    {"type":"straight", "length":100.0},
    {"type":"curved", "curvature":-1/400.0, "angle_in_degrees":30.0},
    {"type":"straight", "length":100.0},
]

# SPECIFY THE NUMERICAL INTEGRATION DETAILS
numerical_integration_parameters = {
    "method" : "rk4",
    "Ts" : 0.05,
    "num_steps_per_Ts" : 1,
}

# SPECIFY THE INITIAL STATE DISTRIBUTION

py_init_min = -1.0
py_init_max =  1.0

v_init_min_in_kmh = 55.0
v_init_max_in_kmh = 65.0

py_init_min = -1.0
py_init_max =  1.0

v_init_min_in_kmh = 55.0
v_init_max_in_kmh = 65.0

initial_state_bounds = {
    "px_init_min" : 0.0,
    "px_init_max" : 0.0,
    "py_init_min" : py_init_min,
    "py_init_max" : py_init_max,
    "theta_init_min" : 0.0,
    "theta_init_max" : 0.0,
    "vx_init_min" : v_init_min_in_kmh * (1.0/3.6),
    "vx_init_max" : v_init_max_in_kmh * (1.0/3.6),
    "vy_init_min" : 0.0,
    "vy_init_max" : 0.0,
    "omega_init_min" : 0.0,
    "omega_init_max" : 0.0,
    "delta_init_min" : 0.0,
    "delta_init_max" : 0.0,
}

# SPECIFY THE TERMINATION PARAMETERS
termination_parameters = {
    "speed_lower_bound"  :  (20.0/3.6),            # to maintain speed above 20 km/h 
    "speed_upper_bound"  :  (200.0/3.6),
    "distance_to_closest_point_upper_bound"  : 20.0,
    "reward_speed_lower_bound"  :  -5000.0,
    "reward_speed_upper_bound"  :  -2000.0,
    "reward_distance_to_closest_point_upper_bound"  :  -5000.0,
}

## Setting Observations: 

In [ ]:
# SPECIFY THE OBSERVATION PARAMETERS
observation_parameters = {
    "should_include_ground_truth_px"                       :  "info",
    "should_include_ground_truth_py"                       :  "info",
    "should_include_ground_truth_theta"                    :  "info",
    "should_include_ground_truth_vx"                       :  "info",
    "should_include_ground_truth_vy"                       :  "info",
    "should_include_ground_truth_omega"                    :  "info",
    "should_include_ground_truth_delta"                    :  "info",
    "should_include_road_progress_at_closest_point"        :  "info",
    "should_include_vx_sensor"                             :  "obs",
    "should_include_distance_to_closest_point"             :  "obs",
    "should_include_heading_angle_relative_to_line"        :  "obs",
    "should_include_heading_angular_rate_gyro"             :  "obs",
    "should_include_closest_point_coords_in_body_frame"    :  "info",
    "should_include_look_ahead_line_coords_in_body_frame"  :  "info",
    "should_include_road_curvature_at_closest_point"       :  "obs",
    "should_include_look_ahead_road_curvatures"            :  "obs",

    "scaling_for_ground_truth_px"                       :  1.0,
    "scaling_for_ground_truth_py"                       :  1.0,
    "scaling_for_ground_truth_theta"                    :  1.0,
    "scaling_for_ground_truth_vx"                       :  1.0,
    "scaling_for_ground_truth_vy"                       :  1.0,
    "scaling_for_ground_truth_omega"                    :  1.0,
    "scaling_for_ground_truth_delta"                    :  1.0,
    "scaling_for_road_progress_at_closest_point"        :  1.0,
    "scaling_for_vx_sensor"                             :  1.0,
    "scaling_for_distance_to_closest_point"             :  1.0,
    "scaling_for_heading_angle_relative_to_line"        :  1.0,
    "scaling_for_heading_angular_rate_gyro"             :  1.0,
    "scaling_for_closest_point_coords_in_body_frame"    :  1.0,
    "scaling_for_look_ahead_line_coords_in_body_frame"  :  1.0,
    "scaling_for_road_curvature_at_closest_point"       :  1.0,
    "scaling_for_look_ahead_road_curvatures"            :  1.0,

    "vx_sensor_bias"    : 0.0,
    "vx_sensor_stddev"  : 0.1,

    "distance_to_closest_point_bias"    :  0.0,
    "distance_to_closest_point_stddev"  :  0.01,

    "heading_angle_relative_to_line_bias"    :  0.0,
    "heading_angle_relative_to_line_stddev"  :  0.01,

    "heading_angular_rate_gyro_bias"    :  0.0,
    "heading_angular_rate_gyro_stddev"  :  0.01,

    "closest_point_coords_in_body_frame_bias"    :  0.0,
    "closest_point_coords_in_body_frame_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_bias"    :  0.0,
    "look_ahead_line_coords_in_body_frame_stddev"  :  0.0,

    "road_curvature_at_closest_point_bias"    :  0.0,
    "road_curvature_at_closest_point_stddev"  :  0.0,

    "look_ahead_road_curvatures_bias"    :  0.0,
    "look_ahead_road_curvatures_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_distance"    :  100.0,
    "look_ahead_line_coords_in_body_frame_num_points"  :  10,
}

## Environment Creation:

In [ ]:
"""class RewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(RewardWrapper, self).__init__(env)

    def step(self, action):
        observation, reward, terminated, truncated, info = self.env.step(action)

        # Create your own reward here
        reward = reward

        return observation, reward, terminated, truncated, info
"""

class RewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(RewardWrapper, self).__init__(env)
        self.previous_progress_at_closest_point = None
        self.previous_heading_angular_rate_gyro = None
        self.previous_heading_angle = None

        # set speed limit
        SPEED_LIMIT = 100.0 / 3.6 # 100 km/h
        self.speed_limit = SPEED_LIMIT 
        
        # constants chosen so that the maximum reward obtained from road proximity and speed are the same order of magnitude (i.e. normalized)
        self.progress_reward_constant = 2.0 # 1.0 
        self.road_proximity_reward_constant_1 = 1.0 
        self.road_proximity_reward_constant_2 = 2.0 
        self.speed_reward_constant = 10.0 # 4.0 
        self.heading_reward_constant_1 = 2.0    
        self.heading_reward_constant_2 = 1.0 # 0.16
        self.angular_velocity_smoothness_reward_constant = 0.20 
        self.high_road_curvature_threshold = 1/250.0 

    def step(self, action):
        observation, reward, terminated, truncated, info = self.env.step(action)

        ###############################################    
        # Custom reward, overwrite the default reward
        ###############################################    
        reward = 0.0 # overwrite default reward

        # set progress reward
        progress_at_closest_point = info["road_progress_at_closest_point"]
        if self.previous_progress_at_closest_point is not None:
            progress_reward = (progress_at_closest_point - self.previous_progress_at_closest_point) * self.progress_reward_constant
        else:
            progress_reward = 0.0
        self.previous_progress_at_closest_point = progress_at_closest_point


        # give higher progress reward for progress closer to the end of the road, to encourage reaching the destination faster
        # also don't give positive road proximity reward if the vehicle is not progressing

        # set road proximity reward, i.e. how close the vehicle is to the road
        #  positive reward if close to the road i.e. within 2m
        #  negative reward if far from the road i.e. more than 2m
        d = abs(observation["distance_to_closest_point"])
        
        road_proximity_reward = 0.0
        if d < 0.5:
            if progress_reward > 0.0:
                road_proximity_reward = 3.5 * self.road_proximity_reward_constant_1 * (1.0 - d**2)

        elif 0.5 <= d < 2.0:
            if progress_reward > 0.0:
                road_proximity_reward = self.road_proximity_reward_constant_1 * (2.0 - d)**2
        else:
            road_proximity_reward = -self.road_proximity_reward_constant_2 * np.tanh((d - 2.0)**4)


        # set speed reward 
        # positive reward if speed is less than speed limit, otherwise negative reward
        # positive reward also reduced in proportion to max look ahead road curvature in the next 3 points 
        vx = abs(observation["vx_sensor"])
        #road_curvature_closest_point = max(np.abs(observation["look_ahead_road_curvatures"][:3]))   
        road_curvature_closest_point = abs(observation["road_curvature_at_closest_point"]) 
        road_curvature_closest_point = max(road_curvature_closest_point, np.max(np.abs(observation["look_ahead_road_curvatures"])))

        # road curvature multiplier to encourage slower speeds on high curvature roads
        if road_curvature_closest_point >= self.high_road_curvature_threshold:
            road_curvature_multiplier = 1.0 /  (1.0 + (400.0*(road_curvature_closest_point - self.high_road_curvature_threshold)**(1.2)))
        else:
            road_curvature_multiplier = 1.0    

        # adaptive speed limit based on road curvature, lower speed limit for higher curvature roads
        adjusted_speed_limit = self.speed_limit * road_curvature_multiplier

        # piecewise linear below speed limit (to encourage driving at a speed closer to the adjusted speed limit)
        if vx < 0.9*adjusted_speed_limit:
            speed_reward =  self.speed_reward_constant * (vx / (0.9*adjusted_speed_limit))**2 

        elif 0.9*adjusted_speed_limit <= vx < adjusted_speed_limit:
            #speed_reward =  -self.speed_reward_constant * (vx - adjusted_speed_limit) / (0.1*adjusted_speed_limit)
            speed_reward =  - 10.0 * self.speed_reward_constant * (vx - adjusted_speed_limit) / (0.1*adjusted_speed_limit)

        else:
            #speed_reward = -self.speed_reward_constant * np.tanh((vx - adjusted_speed_limit)**4)
            #speed_reward = self.speed_reward_constant * ( (1.0/((vx - adjusted_speed_limit+ 1)**4)) - 1.0)
            speed_reward = 0.5 *self.speed_reward_constant * ( (1.0/((vx - adjusted_speed_limit+ 1)**4)) - 1.0)


        # set reward for maintaining small heading angle relative to the line (negative reward proportional to the heading angle)
        # for heading angle smaller than 2 degrees, give positive reward, otherwise give negative reward
        heading_angle_degrees = abs(observation["heading_angle_relative_to_line"]) * 180.0 / np.pi 
        heading_reward = 0.0
        if heading_angle_degrees < 1.0:
            # give positive heading reward only if there is progress
            if progress_reward > 0.0: 
                heading_reward = self.heading_reward_constant_1 * (heading_angle_degrees-1.0)**2 / 4.0
        else:
            heading_reward = -self.heading_reward_constant_2 * abs(heading_angle_degrees)

        # set reward for encouraging smoother steering, by penalize large gradients in heading angle and angular velocity
        if self.previous_heading_angle is not None:
            heading_angle = abs(observation["heading_angle_relative_to_line"])
            heading_angle_change_degrees = abs(heading_angle - self.previous_heading_angle) * 180.0 / np.pi
            angular_velocity_smoothness_reward = -self.angular_velocity_smoothness_reward_constant * heading_angle_change_degrees
        else:
            angular_velocity_smoothness_reward = 0.0
         

        # TODO: check termination condition and set termination rewards
        terminated = False
        termination_reward = 0.0
        if (progress_at_closest_point >= self.env.unwrapped.total_road_length_for_termination):
            #print("(prog,tot_len) = ( " + str(info_dict["progress_at_closest_p"]) + " , " + str(self.total_road_length) + " )" )
            terminated = True
        # > Terminate for being outside of the speed range
        if (vx > self.env.unwrapped.termination_speed_upper_bound):
            terminated = True
            termination_reward += self.env.unwrapped.termination_reward_for_speed_upper_bound
        if (observation["vx_sensor"] < self.env.unwrapped.termination_speed_lower_bound):
            terminated = True
            termination_reward += self.env.unwrapped.termination_reward_for_speed_lower_bound
        # > Terminate for deviating too much from the line
        if (d > self.env.unwrapped.termination_distance_to_closest_point_upper_bound):
            terminated = True
            termination_reward += self.env.unwrapped.termination_reward_for_distance_to_closest_point_upper_bound

        # no truncation
        truncated = False

        # add up all the reward contributions (ignore speed reward for now)
        reward = progress_reward + road_proximity_reward + speed_reward + heading_reward + angular_velocity_smoothness_reward + termination_reward

        return observation, reward, terminated, truncated, info




def create_env(road_elements_list):
    env = gym.make(
        "ai4rgym/autonomous_driving_env",
        render_mode=None,
        bicycle_model_parameters=bicycle_model_parameters,
        road_elements_list=road_elements_list,
        numerical_integration_parameters=numerical_integration_parameters,
        termination_parameters=termination_parameters,
        initial_state_bounds=initial_state_bounds,
        observation_parameters=observation_parameters,
    )

    # Set the integration method and time step
    Ts_sim = 0.05
    integration_method = "rk4"
    env.unwrapped.set_integration_method(integration_method)
    env.unwrapped.set_integration_Ts(Ts_sim)

    # Set the road condition
    env.unwrapped.set_road_condition(road_condition="wet")

    # Rescale actions and wrap the environment
    env = gym.wrappers.RescaleAction(env, min_action=-1, max_action=1)
    env = RewardWrapper(env)
    return env

## Domain Randomization Wrapper:

In [ ]:
class DomainRandomizationWrapper(gym.Wrapper):
    def __init__(self, env, road_randomization_params=None):
        super(DomainRandomizationWrapper, self).__init__(env)
        self.road_randomization_params = road_randomization_params

    def generate_random_road_elements_list(self):
        params = self.road_randomization_params or {}
        num_elements_range = params.get('num_elements_range', (2, 5))
        straight_length_range = params.get('straight_length_range', (50.0, 200.0))
        curvature_range = params.get('curvature_range', (-1/500.0, 1/500.0))
        angle_range = params.get('angle_range', (10.0, 60.0))

        road_elements = []
        num_elements = random.randint(*num_elements_range)

        for _ in range(num_elements):
            element_type = random.choice(['straight', 'curved'])
            if element_type == 'straight':
                length = random.uniform(*straight_length_range)
                road_elements.append({"type": "straight", "length": length})
            else:
                curvature = random.uniform(*curvature_range)
                angle = random.uniform(*angle_range)
                road_elements.append({
                    "type": "curved",
                    "curvature": curvature,
                    "angle_in_degrees": angle
                })
        return road_elements

    def reset(self, **kwargs):
        # Generate a new random road
        self.unwrapped.road_elements_list = self.generate_random_road_elements_list()
        self.unwrapped.road = Road(epsilon_c=(1/10000), road_elements_list=self.unwrapped.road_elements_list)
        self.unwrapped.total_road_length = self.unwrapped.road.get_total_length()
        self.unwrapped.total_road_length_for_termination = max(self.unwrapped.total_road_length - 0.1, 0.9999 * self.unwrapped.total_road_length)
        # Call the original reset method
        return self.env.reset(**kwargs)

### Visualizing Random Roads: 

In [ ]:
# Example 1: Standard Randomization
# road_randomization_params = {
#      'num_elements_range': (3, 6),          # Road will have 3 to 6 elements
#      'straight_length_range': (100.0, 150.0), # Length of straight segments between 100 and 150 meters
#      'curvature_range': (-1/300.0, 1/300.0), # Curvature between -1/300 and 1/300 for curves
#      'angle_range': (20.0, 45.0)            # Angle of the curves between 20 and 45 degrees
#  }

# # Example 2: More Straight Roads
# road_randomization_params = {
#     'num_elements_range': (2, 4),            # Fewer elements, 2 to 4
#     'straight_length_range': (150.0, 300.0), # Longer straight segments between 150 and 300 meters
#     'curvature_range': (-1/1000.0, 1/1000.0), # Small curvature for gentler curves
#     'angle_range': (10.0, 30.0)              # Curves have lower angles between 10 and 30 degrees
# }

# # Example 3: More challenging roads with Tighter curves
# road_randomization_params = {
#       'num_elements_range': (3, 7),            
#       'straight_length_range': (50.0, 150.0), 
#       'curvature_range': (-1/100.0, 1/100.0),  
#       'angle_range': (5.0, 60.0)        
# }

road_randomization_params = {
    'num_elements_range': (3, 10),             # Highly variable number of road elements
    'straight_length_range': (50.0, 300.0),    # Straight segments can be very short or long
    'curvature_range': (-1/50.0, 1/50.0),      # Very tight curves possible
    'angle_range': (10.0, 150.0)                 # Wide range of angles for curves   
}


# # Example 4: Completely Randomized
# road_randomization_params = {
#     'num_elements_range': (1, 10),             # Highly variable number of road elements
#     'straight_length_range': (50.0, 300.0),    # Straight segments can be very short or long
#     'curvature_range': (-1/50.0, 1/50.0),      # Very tight curves possible
#     'angle_range': (10.0, 90.0)                # Wide range of angles for curves
# }

random_env = create_env(road_elements_list = road_elements_list, )
random_env = DomainRandomizationWrapper(random_env, road_randomization_params=road_randomization_params)

# Reset the random env - this will create a new random road
random_env.reset()

# Render the road
random_env.render_matplotlib_init_figure()
random_env.render_matplotlib_plot_road()
plt.show()

## Setting up Training/Evaluation Environments:

In [ ]:
### ------------- MAKE CHANGES TO THE ROAD RANDOMIZATION PARAMS HERE ------- ###
#road_randomization_params = None
road_randomization_params = {
    'num_elements_range': (3, 10),             # Highly variable number of road elements
    'straight_length_range': (50.0, 300.0),    # Straight segments can be very short or long
    'curvature_range': (-1/50.0, 1/50.0),      # Very tight curves possible
    'angle_range': (10.0, 150.0)                 # Wide range of angles for curves   
}

### ------------------------------------------------------------------------ ###

# Create the training environment with domain randomization
train_env = create_env(road_elements_list=road_elements_list)  # Road elements will be randomized
train_env = DomainRandomizationWrapper(train_env, road_randomization_params = road_randomization_params)
train_env = RewardWrapper(train_env)

In [ ]:
# Define a fixed, difficult road for evaluation
road_elements_list_difficult = [
    {"type": "straight", "length": 150.0},
    {"type": "curved", "curvature": 1/100.0, "angle_in_degrees": 110.0},
    {"type": "straight", "length": 50.0},
    {"type": "curved", "curvature": -1/70.0, "angle_in_degrees": 150.0},
    {"type": "straight", "length": 50.0},
    {"type": "curved", "curvature": 1/50.0, "angle_in_degrees": 160.0},
    {"type": "straight", "length": 100.0},
    {"type": "curved", "curvature": -1/200.0, "angle_in_degrees": 60.0},
     
]

# Create the evaluation environment with the fixed difficult road
eval_env = create_env(road_elements_list_difficult)

# Render the road
eval_env.render_matplotlib_init_figure()
eval_env.render_matplotlib_plot_road()
plt.show()

## Model Definition and Training:

In [ ]:
from stable_baselines3 import PPO, SAC, DDPG

model_name = "SAC_dr_wet_long_6"

TIMESTEPS_PER_EPOCH = 50000
NUM_EPOCHS = 20

logdir = "logs"
models_dir = f"models/{model_name}"
figs_dir_train = f"models/{model_name}/figs_train"
figs_dir_test = f"models/{model_name}/figs_test"

ensure_dirs([logdir, models_dir, figs_dir_train, figs_dir_test])

policy_kwargs = dict(net_arch=[256, 256, 256])

# double batch size, double replay buffer size (important for off-policy models), deeper neural network
model = SAC("MultiInputPolicy", train_env, policy_kwargs=policy_kwargs, verbose = 1, tensorboard_log=logdir, batch_size=512, buffer_size=2000000)

def train(start_epoch=1, num_epochs=10, timesteps_per_epoch=50000):
    for i in range(start_epoch, start_epoch+num_epochs):
        model.learn(
            total_timesteps=timesteps_per_epoch,
            reset_num_timesteps=False,
            tb_log_name=f"{model_name}"
        )
        model.save(f"{models_dir}/{timesteps_per_epoch * i}")
        # Evaluate on the most recent training road
        eval_model(train_env, model, figs_dir_train, timesteps_per_epoch * i)
        # Evaluate on evaluation road
        eval_model(eval_env, model, figs_dir_test, timesteps_per_epoch * i)

    start_epoch += num_epochs    
    return start_epoch

In [ ]:
start_epoch = 1
num_epochs = 50
timesteps_per_epoch = 2000

start_epoch = train(start_epoch, num_epochs, timesteps_per_epoch)

## Evaluating the RL Policy:

### Load an RL model from the training loop:

In [ ]:
model_name = "SAC_dr_wet_long_6"
model_idx = 100000

path_for_saving_figures = f"models/{model_name}/eval"
ensure_dir(path_for_saving_figures)

### Put the RL model into a policy class with a 'standardize' interface

In [ ]:
from policies.rl_policy import RLPolicy

# Put the RL model into a policy class
rl_policy = RLPolicy(model)

### Perform One simulation of the policy, plot the time series results:

In [ ]:
# Import the function for simulating the autonomous driving environment
from evaluation.evaluation_for_autonomous_driving import simulate_policy
from evaluation.evaluation_for_autonomous_driving import plot_results_from_time_series_dict

# Specify the length of the simulation in time steps
# > If termination or truncation is flagged by the "step" function,
#   Then, the simulation ends.
N_sim = 30000

# Specify the seed for when the simulate function resets the random number generator
sim_seed = 1;

# Call the function for simulating a given RL model
sim_time_series_dict = simulate_policy(eval_env, N_sim, rl_policy, seed=sim_seed, verbose=1)

# Call the plotting function
file_name_suffix = "example"
plot_details_list = plot_results_from_time_series_dict(eval_env, sim_time_series_dict, path_for_saving_figures, file_name_suffix, should_plot_reward=True)

### Animate the time series results:

In [ ]:
"""def animate_from_sim_time_series_dict(sim_time_series_dict, Ts, path_for_saving_figures):
    # Extract the necessary trajectory information from the "sim_time_series_dict"
    px_traj    = sim_time_series_dict["px"]
    py_traj    = sim_time_series_dict["py"]
    theta_traj = sim_time_series_dict["theta"]
    delta_traj = sim_time_series_dict["delta"]
    # Call the environments function to create the simulation
    ani = eval_env.unwrapped.render_matplotlib_animation_of_trajectory(px_traj, py_traj, theta_traj, delta_traj, Ts, traj_increment=3)
    # Save the animation
    ani.save(f"{path_for_saving_figures}/trajectory_animation.gif")
    print(f'Saved animation at {path_for_saving_figures}/trajectory_animation.gif')
    # Return the animation object
    return ani"""

In [ ]:
"""# Call the animation functoin
ani = animate_from_sim_time_series_dict(sim_time_series_dict, numerical_integration_parameters["Ts"], path_for_saving_figures)
# Display the animation
from IPython.display import HTML
HTML(ani.to_jshtml())"""

## Performance Metrics:

### Define a performance Metrics Function per simulation time series:

In [ ]:
def compute_performance_metrics_from_time_series(sim_time_series_dict):
    # Compute the statistics of the distance to the line
    # > Note that this is signed value, hence need to take absolute value of the data.
    abs_dist_to_line_time_series = np.abs(sim_time_series_dict["distance_to_closest_point"])
    avg_dist  = np.nanmean(abs_dist_to_line_time_series)
    std_dist  = np.nanstd(abs_dist_to_line_time_series)
    max_dist  = np.nanmax(abs_dist_to_line_time_series)

    # Compute the statistics of the speed in the forward direction (i.e., the body-frame x-axis direction)
    speed_time_series = np.abs(sim_time_series_dict["vx"])
    avg_speed = np.nanmean(speed_time_series)
    std_speed = np.nanstd(speed_time_series)
    max_speed = np.nanmax(speed_time_series)
    min_speed = np.nanmin(speed_time_series)

    # Return the results
    return {
        "avg_dist"   :  avg_dist,
        "std_dist"   :  std_dist,
        "max_dist"   :  max_dist,
        "avg_speed"  :  avg_speed * 3.6,
        "std_speed"  :  std_speed * 3.6,
        "max_speed"  :  max_speed * 3.6,
        "min_speed"  :  min_speed * 3.6,
    }


### Run the Performance Metric Function for one simulation:

In [ ]:
# Simulate with the same seed as above to check that it gets the results consistent
sim_seed = 1;
sim_time_series_dict = simulate_policy(eval_env, N_sim, rl_policy, seed=sim_seed, verbose=1)

# Call the function for computing the performance metrics
pm_dict = compute_performance_metrics_from_time_series(sim_time_series_dict)

# Display the performance metric values
print("Performance Metric dictionary:")
print(pm_dict)

**NOTE:**
> Please refer to the notebook `autonomous_driving_gym_v2024_08_24.ipynb` for more details on evaluation and performance metrics. 